# Overview

For this project, our group decided to perform an ETL process for Trending Youtube Video Statistics. Our datasets included daily trending Youtube videos in the US and CA regions as well as a dataset of various categories the videos belong to. Our ETL process seeks to read the datasets, clean up the dataset according to our standards, and load into a database. From there, we determine which video/category is the most/least popular with respect to the number of likes, dislikes, views, and comments.   

In [ ]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import username, password

## Extraction

We downloaded our data from Kaggle through the following link: https://www.kaggle.com/datasnaek/youtube-new

The files we specifically utilized were daily trending Youtube videos in the US and CA regions (formatted as .csv) as well as category files (formatted as .json) that details the different categories the videos belong to. The links to the files are provided below

     - https://www.kaggle.com/datasnaek/youtube-new?select=CAvideos.csv
     - https://www.kaggle.com/datasnaek/youtube-new?select=USvideos.csv
     - https://www.kaggle.com/datasnaek/youtube-new?select=CA_category_id.json
     - https://www.kaggle.com/datasnaek/youtube-new?select=US_category_id.json


These files are read by the Pandas library in Jupyter Notebook for the transformation step. 

In [ ]:
#Read resources
CA_videos = pd.read_csv('Resources/CAvideos.csv')
US_videos = pd.read_csv('Resources/USvideos.csv')
CA_category_id = pd.read_json('Resources/CA_category_id.json')
US_category_id = pd.read_json('Resources/US_category_id.json')

In [ ]:
#Jumaan start here

## Transformation

The Pandas library was the main tool to perform data clean up/transformation. It was done so as following:

    - Read the CAvideos.csv and USvideos.csv files into a pandas dataframe.
    - Drop any unnecessary columns and rename columns for improved readability. 
    - Check for any duplicate rows that exist within the dataframe and drop them.
    - Read the CA_category_id.json and US_category_id.json files into a pandas dataframe.
    - Only the category_id and category_title information is necessary, so extract that from the "items" column of the category dataframes. 
    - Since the "items" column exists as a list, iterate over the list and append the various ids and titles in separate lists in order to create a new data frame.
    - Finally, merge the CA and US videos dataframe with their respective category dataframe on category_id. Once merged, add a column for country that specifies either "US" or "CA" for each dataframe. 

In [ ]:
#Check the columns on US_videos
US_videos.head()

In [ ]:
#Drop unnecessary columns and rename
US_videos_new = US_videos[['title', 'category_id', 'views', 'likes', 'dislikes', 'comment_count']]

US_videos_new.rename(columns={"comment_count":"comments"},inplace= True)

US_videos_new.head()

In [ ]:
#Check for duplicates and drop any duplicate rows
print(len(US_videos_new)) 
print(len(US_videos_new['title'].unique()))

US_videos_new.drop_duplicates(subset=["title"],keep = "last",inplace=True)

print(len(US_videos_new))

In [ ]:
#Check the columns on US_category_id
US_category_id.head()

In [ ]:
#Turn items column into a list
US_category_id_list = US_category_id['items'].tolist()

#Loop through the list and extract category id and title
category_id = []
category_title = []

for category in US_category_id_list:
    category_id.append(int(category['id']))
    category_title.append(category['snippet']['title'])

#Store the information in a datagrame
US_categories = pd.DataFrame({"category_title": category_title, "category_id": category_id})

US_categories.head()

len(US_categories)

In [ ]:
#Merge both dataframes on category_id
US_merged = pd.merge(US_videos_new, US_categories, on="category_id", how="inner")
US_merged.head()

In [ ]:
#Jumaan end here

In [ ]:
#Shrey start here

In [ ]:
#Reading the CAvideo file into a dataframe
CA_videos.head()

In [ ]:
#Dropping the unnecessary columns
CA_videos_new = CA_videos[["title","category_id","views","likes","dislikes","comment_count"]].copy()
CA_videos_new.head()

In [ ]:
#Renaming the columns
CA_videos_new.rename(columns={"comment_count":"comments"},inplace= True)
CA_videos_new.head()

In [ ]:
len(CA_videos_new["title"].unique())

In [ ]:
len(CA_videos_new)

In [ ]:
#Dropping the duplicates
CA_videos_new.drop_duplicates(subset=["title"],keep = "last",inplace=True)

In [ ]:
len(CA_videos_new)

In [ ]:
CA_category_id.head()

In [ ]:
len(CA_category_id)

In [ ]:
CA_category_id["items"][0]

In [ ]:
items_list = CA_category_id["items"].tolist()

In [ ]:
#Extracting the required information from the category dataframe
id_list = []
title=  []
for item in items_list:
    id_list.append(int(item["id"]))
    title.append(item["snippet"]["title"])

In [ ]:
#Storing the information in a dataframe
new_category_df = pd.DataFrame({"category_title":title,"category_id":id_list})

In [ ]:
new_category_df.head()

In [ ]:
len(new_category_df)

In [ ]:
new_category_df.dtypes

In [ ]:
new_category_df["category_id"] = new_category_df["category_id"].astype(int)
new_category_df.dtypes

In [ ]:
#Merging the individual country dataframes with the category dataframe
mergedCA_df = pd.merge(CA_videos_new,new_category_df,on="category_id",how="inner")
mergedCA_df
mergedCA_df["country"] = "CA"
mergedCA_df.tail()

In [ ]:
mergedUS_df = pd.merge(US_videos_new,new_category_df,on="category_id",how="inner")
mergedUS_df.dropna()
mergedUS_df["country"] = "US"
mergedUS_df.head()

In [ ]:
#Anji start here

# Loading

We decided to use PostgreSQL (relational database) to load our data

__Connect to local database__


In [ ]:
rds_connection_string = f'{username}:{password}@localhost:5432/youtube_db'
engine = create_engine(f'postgresql://{rds_connection_string}')

__Check for tables__

In [ ]:
engine.table_names()

__Use PANDAS to load csv/json converted DataFrames into database__

In [ ]:
# US dataframe
mergedUS_df.to_sql(name="merged_us", con=engine, if_exists="append", index=False)

In [ ]:
# CA dataframe
mergedCA_df.to_sql(name="merged_ca", con=engine, if_exists="append", index=False)

__Confirm data has been added by querying both tables__

In [ ]:
pd.read_sql_query("select * from merged_us", con=engine).head()

In [ ]:
pd.read_sql_query("select * from merged_ca", con=engine).head()

In [ ]:
#Anji end here